# Reference 
https://huggingface.co/joeddav/bart-large-mnli-yahoo-answers
https://huggingface.co/facebook/bart-large-mnli

In [ ]:
!pip install datasets
!pip install transformers
!pip install -U scikit-learn
!pip install sentencepiece
!pip install openpyxl
!pip install pyvi

In [ ]:
!pip install gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1PpR36GRQR26nRysMkgOz2S870dspBMyY  # for files
# gdown <file_id>                                 # alternative format
# gdown --folder https://drive.google.com/drive/folders/<file_id>  # for folders
# gdown --folder --id <file_id>                                   # this format works for folders too

In [ ]:
!ls -la

In [ ]:
!unzip "/content/data.zip" -d "/content/"

In [ ]:
import re
import string

import pandas as pd
from pyvi import ViTokenizer
from sklearn.model_selection import train_test_split

def remove_stopwords(tokens):
    stopwords = open("stopwords.txt").read().split("\n")
    tokens = [i.replace("_", " ") for i in tokens]
    tokens = [i for i in tokens if i not in stopwords]
    text = " ".join(i for i in tokens)
    return text


def remove_punctuation(text):
    text = text.lower()
    table = str.maketrans({key: None for key in string.punctuation})
    text = text.translate(table)
    text = re.sub('\s+', ' ', text)
    return text


def normalize(text):
    # tokens = ViTokenizer.tokenize(text).split()
    tokens = ViTokenizer.tokenize(text)
    text = tokens
    # text = remove_stopwords(tokens)
    # text = remove_punctuation(text)
    return text
from datasets import Dataset

def load_dataset(path):
    df = pd.read_excel(path)
    X = list(df["text"])
    X = [normalize(x) for x in X]
    # y = df.drop("text", axis=1)
    # columns = y.columns
    # temp = y.apply(lambda item: item > 0)
    # y = list(temp.apply(lambda item: list(columns[item.values]), axis=1))
    # return X, y
    df['text']=X
    y = df.drop("text", axis=1)
    columns = y.columns
    temp = y.apply(lambda item: item > 0)
    column_labels = df.columns.drop('text')
    df['label']=y.apply(lambda row: list(row.values), axis=1)
    # df['label_text']=list(temp.apply(lambda item: list(columns[item.values]), axis=1))

    dataset = Dataset.from_pandas(df.drop(column_labels, axis=1))
    dataset=dataset.train_test_split(test_size=0.2)
    return dataset


In [ ]:
import datasets
import torch
from transformers import AutoModel, AutoTokenizer

import pathlib
import os
from datasets import Dataset



# https://huggingface.co/docs/transformers/tasks/token_classification
#BARTpho-word
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word")
TXT = 'Chúng tôi là những nghiên cứu viên.'
input_ids = tokenizer(TXT, truncation=True)
input_ids = tokenizer(TXT, return_tensors='pt')
# print(TXT, input_ids)
# input_ids = tokenizer(TXT, is_split_into_words=True)
# tokens = tokenizer.convert_ids_to_tokens(input_ids['input_ids'])
# print(tokens)
features = bartpho_word(input_ids['input_ids'])
# print(features)

In [ ]:
bartpho_word

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True,truncation=True, max_length=512 )
# , return_tensors='pt', max_length=512, truncation=True


In [ ]:
# train_path = os.path.join(pathlib.Path(__file__).parent.parent.resolve(), 'train.xlsx')
# print("Load data", train_path)
# dataset = load_dataset(train_path)
# dataset.save_to_disk("../data/data_train")
dataset = datasets.load_from_disk("/content/data/data_train")

In [ ]:
dataset['train']

In [ ]:
dataset

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print('dataset', tokenized_dataset['train'][0])
tokenized_dataset

In [ ]:
tokenized_dataset['train']

In [ ]:
len(tokenized_dataset['train']['input_ids'][0])

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print('data_collator sample')
# tokenized_dataset = tokenized_dataset.remove_columns(dataset["train"].column_names)
# tokenized_dataset = tokenized_dataset.remove_columns(dataset["test"].column_names)

In [ ]:
len(tokenized_dataset["train"])

In [ ]:
print('data loaded....................')
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, MBartForSequenceClassification
model = MBartForSequenceClassification.from_pretrained("vinai/bartpho-word", num_labels=10)
# model = AutoModelForSequenceClassification.from_pretrained("vinai/bartpho-word", num_labels=10)

print('load model AutoModelForSequenceClassification')
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)
print('load model Trainner')
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
print('load model trainer.train()')
trainer.train()

In [ ]:
trainer.save_model("bart_class_train")

In [ ]:
################################################################################################################################